# Continuation Power Flow

Demonstrates continuation power flow (CPF) analysis for voltage stability
assessment. The notebook sets up interface transfers, runs the CPF solver
to trace the PV curve, and visualizes the nose point that marks the voltage
stability limit.

In [ ]:
import numpy as np
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Area

In [2]:
# This cell is hidden in the documentation.
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 4.2377 sec


In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_pv_curve

## 1. Continuation Power Flow

The `continuation_pf()` method iteratively increases interface flow and tracks
voltages until a bifurcation (voltage collapse) point is reached. It yields
the interface MW at each stable solution.

In [3]:
# Solve base case
V_base = wb.pflow()
print(f"Base case min voltage: {np.abs(V_base).min():.4f} pu")

# Track voltage at critical bus during CPF
critical_bus_idx = np.argmin(np.abs(V_base))
print(f"Critical bus index: {critical_bus_idx}")
print(f"Critical bus voltage: {np.abs(V_base[critical_bus_idx]):.4f} pu")

Base case min voltage: 0.9749 pu
Critical bus index: 10
Critical bus voltage: 0.9749 pu


## 2. PV Curve

The PV curve shows how voltage at a critical bus varies with increasing
power transfer. The nose point indicates the maximum transfer before
voltage collapse.

In [ ]:
# Collect PV curve data points
mw_points = []
v_points = []

try:
    for mw in wb.stat.continuation_pf(
        interface=None,
        initialmw=0,
        minstep=1,
        maxstep=50,
        maxiter=200
    ):
        V = wb.voltage()
        v_critical = np.abs(V[critical_bus_idx])
        mw_points.append(mw)
        v_points.append(v_critical)
except Exception as e:
    print(f"CPF terminated: {e}")

print(f"Collected {len(mw_points)} points")
if mw_points:
    print(f"Transfer range: {min(mw_points):.1f} to {max(mw_points):.1f} MW")

CPF terminated: 'GridWorkBench' object has no attribute 'statics'
Collected 0 points


In [ ]:
plot_pv_curve(mw_points, v_points)